In [ ]:
import pandas as pd

# Load the csv file
df = pd.read_csv('lab_manual2.csv', sep=',')
df.columns = df.columns.str.strip()
print(df.columns)



# convert time to integer milliseconds
df['time'] = (df['time'] * 1000).astype(int)

# Reorder the columns
# df = df[['time', 'px', 'py', 'pz', 'qw', 'qx', 'qy', 'qz']]


# Write the DataFrame back to a new csv file with the reordered columns
df.to_csv('imu_output.csv', index=False)

#python process_result.py lab-bumper1

In [1]:
import pandas as pd

# Load the dataframe from csv
with open('lab_motion4_tum.csv', 'r') as f:
    current_content = f.read()

# Open the file in write mode (this will overwrite it)
with open('lab_motion4_tum.csv', 'w') as f:
    # Write the header
    f.write('time px py pz qx qy qz qw\n')
    # Write the original content after the header
    f.write(current_content)
    
df = pd.read_csv('lab_motion4_tum.csv', sep=' ')
df.columns = df.columns.str.strip()
print(df.columns)

# Open the txt file (or create it)
with open('stamped_traj_estimate.txt', 'w') as f:
    # Write the header
    f.write("#timestamp tx ty tz qx qy qz qw\n")

    df['time'] = df['time'].astype(float)


    # Write the data in scientific notation
    for index, row in df.iterrows():
        timestamp = '{:.12f}'.format(row['time'])
        f.write("{} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f}\n".format(
            timestamp, row['px'], row['py'], row['pz'], row['qx'], row['qy'], row['qz'], row['qw']))

print("stamped_traj_estimate.txt")


Index(['time', 'px', 'py', 'pz', 'qx', 'qy', 'qz', 'qw'], dtype='object')
stamped_traj_estimate.txt


In [2]:
import pandas as pd

# Load the dataframe from csv
# df = pd.read_csv('lab_bumper1_gt.csv', delimiter=",")
# df.columns = df.columns.str.strip()
# print(df.columns)
# print(df.head)

def extract(gt,outputname):
    # Read the CSV file using pandas
    df = pd.read_csv(gt, delimiter=" ", header=0)

    # Open the output file
    with open(outputname, 'w') as fout:
        # Write the header
        fout.write('# time tx ty tz qx qy qz qw\n')

        # Iterate over the rows of the DataFrame
        for index, row in df.iterrows():
            # Format the output line
            # line = "{:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f}\n".format(
            #     row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])   
            line = "{:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f} {:.12f}\n".format(
            row[0], row[1], row[2], row[3], row[4], row[5], row[6], row[7])

            # Write the line to the output file
            fout.write(line)

extract('lab_motion4_gt.csv','stamped_groundtruth.txt' )
print("Data written to stamped_groundtruth.txt")


Data written to stamped_groundtruth.txt


In [3]:
import numpy as np
import pandas as pd
from scipy.spatial.transform import Rotation as R

# Provided transformation matrix
T = np.array([[0, 0, -1, 0], 
              [0, 1, 0, 0], 
              [1, 0, 0, 0], 
              [0, 0, 0, 1]])

T1 = np.array([[0,0,-1],
               [0,1,0],
               [1,0,0]])

def transform_positions(input_file_name, output_file_name):
    # Read the csv file, skipping the first row
    df = pd.read_csv(input_file_name, sep=' ', 
                     names=['timestamp', 'tx', 'ty', 'tz', 'qx', 'qy', 'qz', 'qw'], skiprows=1)

    # Initialize a new dictionary to hold transformed positions
    transformed_data = {'timestamp': [], 'tx': [], 'ty': [], 'tz': [], 'qx': [], 'qy': [], 'qz': [], 'qw': []}

    # Apply transformation to each position
    for index, row in df.iterrows():
        # Original position
        original = np.array([row['tx'], row['ty'], row['tz'], 1])

        # Apply transformation
        transformed = np.matmul(T, original)

        # Extract new position
        new_position = transformed[:3]


        # Original rotation as quaternion
        original_rotation = R.from_quat([row['qx'], row['qy'], row['qz'], row['qw']])

        # Convert quaternion to rotation matrix
        original_rotation_matrix = original_rotation.as_matrix()

        # Apply T1 to the rotation matrix
        transformed_rotation_matrix = np.matmul(T1, original_rotation_matrix)

        # Convert transformed rotation matrix back to quaternion
        transformed_rotation = R.from_matrix(transformed_rotation_matrix)
        transformed_rotation_quat = transformed_rotation.as_quat()

        # Add new position and rotation to the dictionary
        transformed_data['timestamp'].append(row['timestamp'])
        transformed_data['tx'].append(new_position[0])
        transformed_data['ty'].append(new_position[1])
        transformed_data['tz'].append(new_position[2])
        transformed_data['qx'].append(transformed_rotation_quat[0])
        transformed_data['qy'].append(transformed_rotation_quat[1])
        transformed_data['qz'].append(transformed_rotation_quat[2])
        transformed_data['qw'].append(transformed_rotation_quat[3])

    # Convert the dictionary to a DataFrame
    transformed_df = pd.DataFrame(transformed_data)

    # Write new positions to a csv file
    transformed_df.to_csv(output_file_name, index=False, sep=' ')

# Apply transformation to both files
# transform_positions('stamped_traj_estimate.txt', 'stamped_traj_estimate.txt')
transform_positions('stamped_groundtruth.txt', 'stamped_groundtruth.txt')
print('done')


done
